<a href="https://colab.research.google.com/github/CorralesK/Lab04_EDA/blob/main/lab04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Laboratorio 04 - EDA**

In [ ]:
# Cargando las librerias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import skew, kurtosis

## 1. Carga y comprensión inicial de los datos

In [ ]:
# Ruta del archivo
filepath = 'https://raw.githubusercontent.com/CorralesK/Lab04_EDA/main/datos.csv' # Me falta que se cargue de git

# Cargar el dataset
df = pd.read_csv(filepath)

In [28]:
# Verificar el formato de las columnas. Comprobar el tipo de cada
print("Información del dataset:")
df.info()

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   churn                       5000 non-null   string 
 1   accountlength               5000 non-null   int64  
 2   internationalplan           5000 non-null   string 
 3   voicemailplan               5000 non-null   string 
 4   numbervmailmessages         5000 non-null   int64  
 5   totaldayminutes             5000 non-null   float64
 6   totaldaycalls               5000 non-null   int64  
 7   totaldaycharge              5000 non-null   float64
 8   totaleveminutes             5000 non-null   float64
 9   totalevecalls               5000 non-null   int64  
 10  totalevecharge              5000 non-null   float64
 11  totalnightminutes           5000 non-null   float64
 12  totalnightcalls             5000 non-null   int64  
 13  totalnig

In [ ]:
# Explorar las primeras filas
print(df.head())

In [ ]:
# Observar la cantidad de filas y columnas
print(f"Dimensiones del dataset (filas, columnas): {df.shape}")

## 2. Limpiar y preparar los datos

In [ ]:
# Identificar y tratar valores nulos
print("Valores nulos por columna:")
df.isnull().sum()

In [ ]:
# Eliminar duplicados
initial_rows = df.shape[0]
df = df.drop_duplicates()
print(f"Filas eliminadas por duplicados: {initial_rows - df.shape[0]}")

In [ ]:
# Revisar valores atípicos

# Función reutilizable para identificar atípicos con el método IQR.
def identificar_atipicos(data, columna, factor=1.5):
    Q1 = data[columna].quantile(0.25)
    Q3 = data[columna].quantile(0.75)
    IQR = Q3 - Q1
    mediana = data[columna].median()
    valor_minimo = data[columna].min()
    valor_maximo = data[columna].max()

    print(f"\nResumen estadístico para {col}:")
    print(f"- Primer cuartil (Q1): {Q1}")
    print(f"- Tercer cuartil (Q3): {Q3}")
    print(f"- Rango intercuartílico (IQR): {IQR}")
    print(f"- Mediana: {mediana}")
    print(f"- Valor mínimo: {valor_minimo}")
    print(f"- Valor máximo: {valor_maximo}\n")

    # Sacar cuales son los limites
    limite_inferior = Q1 - (factor * IQR)
    limite_superior = Q3 + (factor * IQR)

    return data[(data[columna] < limite_inferior) | (data[columna] > limite_superior)]

In [ ]:
# - Identificar atípicos en todas las columnas numéricas
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    atipicos = identificar_atipicos(df, col)
    print(f"- Valores atípicos: {len(atipicos)} registros")
    if len(atipicos) > 0:
        print(f"- Rango de valores atípicos: {atipicos[col].min()} a {atipicos[col].max()}")

In [ ]:
# Convertir tipos de datos
# - Identificar las columnas y sus tipos actuales
df.dtypes

In [ ]:
# - Convertir tipos de datos
df['churn'] = df['churn'].astype('string')
df['internationalplan'] = df['internationalplan'].astype('string')
df['voicemailplan'] = df['voicemailplan'].astype('string')

In [ ]:
# - Verificar los tipos de datos
df.dtypes

In [ ]:
# Cambiar nombre de columnas si procede

# No es necesario cambiar nombres, pero si fuera el caso:
# df.rename(columns={"old_name": "new_name"}, inplace=True)
print("Nombres de columnas actuales:")
df.columns

In [ ]:
# Tratar con variables categóricas
categorical_cols = df.select_dtypes(include='string').columns

print("Variables categóricas identificadas:", categorical_cols.tolist())

for col in categorical_cols:
    print(f"\nDistribución de {col}:")
    print(df[col].value_counts())

## 3. Análisis univariado

In [ ]:
# Estadísticas descriptivas
df.describe()

In [ ]:
# Histograma y distribución de datos
num_cols = df.select_dtypes(include=['float64', 'int64']).columns
for col in num_cols:
    plt.figure(figsize=(14, 6))

    # Histograma con KDE
    plt.subplot(1, 2, 1)
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f"Distribución de {col}")

    # Boxplot
    plt.subplot(1, 2, 2)
    sns.boxplot(x=df[col])
    plt.title(f"Boxplot de {col}")

    plt.tight_layout()
    plt.show()

In [ ]:
# Comprobación de asimetría y normalidad
for col in num_cols:
    skewness = skew(df[col])
    kurt = kurtosis(df[col])
    print(f"{col}:")
    print(f"  Asimetría (Skewness): {skewness:.2f}")
    print(f"  Curtosis: {kurt:.2f}")

## 4. Análisis bivariado